### **[LeetCode Link](https://leetcode-cn.com/problems/find-the-kth-smallest-sum-of-a-matrix-with-sorted-rows/solution/bao-li-jie-fa-zui-xiao-dui-by-coldme-2/)**

## 暴力解法
### 解题思路
* 将“求前m行的第k个最小数组和”按动态规划的思路划分成子问题：已知前m-1行的最小数组和的列表，求前m行的第k个最小数组和。
* 例如，`mat = [[1,10,10],[1,4,5],[2,3,6]]`，`k = 7`。已知前两行的最小数组和 `last_row = [2,5,6,11,11,14,14,15,15]`。我们截取last_row的前k个元素（因为后面的元素不再有竞争力），`last_row = [2,5,6,11,11,14,14]`。然后遍历第三行的每个元素，与last_row的每个元素求和，排序+截取后得到 `new_row = [4, 5, 7, 8, 8, 8, 9]`。返回第k个元素即可。
### 复杂度分析
* 时间复杂度：$O(m*(nk+nklog(nk)) = O(mnklog(nk))$。遍历每行，时间复杂度为 $O(m)$。对于每一行，遍历每一列的元素并求和为 $O(nk)$，对该行求出的数组和（长度不超过nk）排序的复杂度为 $O(nklog(nk))$。（经评论提示后已修改）
空间复杂度：$O(k)$。每行都要存储一个长度不超过k的"数组和"列表。

In [ ]:
class Solution:
    def kthSmallest(self, mat: List[List[int]], k: int) -> int:
        last_row = [0]
        for i in range(len(mat)):
            new_row = [] 
            for j in range(len(mat[0])):
                for p in last_row:
                    new_row.append(p + mat[i][j])
            new_row.sort()
            if len(new_row) > k:
                last_row = new_row[ :k]
            else:
                last_row = new_row
        return last_row[k - 1]

## 最小堆
### 解题思路
每次从堆中弹出最小的数组和`curr_sum`和对应的指针`pointers`，然后轮流将指针`pointers`的每个索引向后移动一位，生成新的`new_sum`，加入堆中。
### 算法流程
* 最小堆存储的是`[curr_sum, pointers]`二元组，`pointers`是指针数组，`curr_sum`是该`pointers`指向的元素的和。初始化`pointers`全为0，求出相应的`curr_sum`，并将其入堆。
* 重复下列步骤k次：
 * 从堆中`pop`出`curr_sum`和`pointers`。
 * 遍历`pointers`的每个索引，将该索引加一，求出新的和，如果没有出现过，`push`入堆。

### 栗子
* 以`mat = [[1,10,10],[1,4,5],[2,3,6]]`，`k = 7`为例，初始化`pointers = (0, 0, 0)，curr_sum = 4`，哈希表`seen`加入`(0, 0, 0)`
* 从堆中`pop`出最小和，`pointers = (0, 0, 0)，curr_sum = 4`。新生成`[13, (1, 0, 0)],[7, (0, 1, 0)]，[5, (0, 0, 1)]`，在`seen`中做好标记，然后将三者入堆。重复该步骤k次。

### 复杂度分析
* 时间复杂度：$O(kmlog(k))$。堆中的元素个数不会超过 $km$个。执行$k$次循环：每次循环时，出堆操作，是$O(log(mk)))$；每次出堆后，要生成$m$个新的数组和，是$O(m)$；将新生成的的数组和入队，是$O(log(mk))$。所以总的是$O(k * (log(km) + m * log(mk))) = O(kmlog(mk)))$ 。
* 空间复杂度：$O(km^2)$。堆中的元素个数不会超过 $km$个，每个的空间是$O(m)$

In [ ]:
import heapq

class Solution:
    def kthSmallest(self, mat, k: int) -> int:
        m, n = len(mat), len(mat[0])
        # 初始化指针
        pointers = [0] * m 
        # 初始化heap
        heap = []
        curr_sum = 0
        for i in range(m):
            curr_sum += mat[i][0]
        heapq.heappush(heap, [curr_sum, tuple(pointers)])
        # 初始化seen
        seen = set()
        seen.add(tuple(pointers))
        # 执行k次
        for _ in range(k):
            # 从堆中pop出curr_sum(最小数组和)和pointers(指针数组)
            curr_sum, pointers = heapq.heappop(heap)
            # 每个指针轮流后移一位，将new_sum(新的数组和)和new_pointers(新的指针数组)push入堆
            for i, j in enumerate(pointers):
                if j < n - 1:
                    new_pointers = list(pointers)
                    new_pointers[i] = j + 1
                    new_pointers = tuple(new_pointers)
                    if new_pointers not in seen:
                        new_sum = curr_sum + mat[i][j + 1]- mat[i][j]
                        heapq.heappush(heap, [new_sum, new_pointers])
                        seen.add(new_pointers)
        return curr_sum